In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import classification_report
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import matplotlib.pyplot as plt 
import seaborn as sn 
from sklearn.model_selection import train_test_split

c:\Users\Kelvin Brachthuizen\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Read data
own_data = pd.read_csv('data/own_dataset.tsv', sep='\t')
test_data = pd.read_csv('data/sentiment-topic-test.tsv', sep='\t')

#remove sentiment column and sentence id column
own_data = own_data.drop(columns=['sentiment'])
own_data = own_data.drop(columns=['sentence id'])
test_data = test_data.drop(columns=['sentiment'])
test_data = test_data.drop(columns=['sentence id'])

#change each value in topic to an integer
own_data['topic'] = own_data['topic'].astype('category')
own_data['topic'] = own_data['topic'].cat.codes
test_data['topic'] = test_data['topic'].astype('category')
test_data['topic'] = test_data['topic'].cat.codes

#change topic header to labels
own_data = own_data.rename(columns={"topic": "labels"})
test_data = test_data.rename(columns={"topic": "labels"})

In [3]:
all_data = pd.concat([own_data, test_data])

In [4]:
#split data
train, test = train_test_split(own_data, test_size=10, shuffle=False)
train, dev = train_test_split(train, test_size=0.3, shuffle=True)


In [5]:
# Model configuration # https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model 
model_args = ClassificationArgs()

model_args.overwrite_output_dir=True # overwrite existing saved models in the same directory
model_args.evaluate_during_training=True # to perform evaluation while training the model
# (eval data should be passed to the training method)

model_args.num_train_epochs=15 # number of epochs
model_args.train_batch_size=64 # batch size
model_args.learning_rate=4e-5 # learning rate
model_args.max_seq_length=256 # maximum sequence length
# Note! Increasing max_seq_len may provide better performance, but training time will increase. 
# For educational purposes, we set max_seq_len to 256.

# Early stopping to combat overfitting: https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model_args.use_early_stopping=True
model_args.early_stopping_delta=0.01 # "The improvement over best_eval_loss necessary to count as a better checkpoint"
model_args.early_stopping_metric='eval_loss'
model_args.early_stopping_metric_minimize=True
model_args.early_stopping_patience=2
model_args.evaluate_during_training_steps=32 # how often you want to run validation in terms of training steps (or batches)

In [6]:
bert_model = ClassificationModel('bert', 'bert-base-cased', num_labels=3, args=model_args, use_cuda=True) # CUDA is enabled
_, history = bert_model.train_model(train, eval_df=dev, multi_label=False) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
1it [00:08,  8.06s/it]
Epochs 1/15. Running Loss:    1.1860: 100%|██████████| 1/1 [14:10<00:00, 850.50s/it]
1it [00:07,  7.89s/it]
Epochs 2/15. Running Loss:    1.1748: 100%|██████████| 1/1 [01:13<00:00, 73.62s/it]
1it [00:07,  7.48s/it]
Epochs 3/15. Running Loss:    1.0593: 100%|██████████| 1/1 [00:37<00:00, 37.13s/it]
1it [00:07,  7.20s/it]
Epochs 4/15. Running Loss:    1.0322: 100%|██████████| 1/1 [00:37<00:00, 37.36s/it]
1it [00:07,  7.23s/it]
Epochs 5/15. Running Loss:    0.9660: 100%|██████████| 1/1 [00:37<00:00, 37.26s/it]
1it [00:07,  7.21s/it]
Epochs 6/15. Running Loss:    0.9049: 100%|██████████| 1/1 [00:36<00:00, 36.49s/it]
1it [00:07,  7.58s/it]
Epochs 7/15. Running Loss:    0.8431: 100%|

In [9]:
def evaluate_model(model: ClassificationModel, dev: pd.DataFrame, test:pd.DataFrame):
    result, outputs, wrong_pred = model.eval_model(dev)
    
    predicted, probabilities = model.predict(test.text.to_list())
    test['predicted'] = predicted
    
    print(classification_report(test['labels'], test['predicted']))

evaluate_model(bert_model, dev, test)

1it [00:07,  7.45s/it]
Running Evaluation: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it]
1it [00:07,  7.15s/it]
100%|██████████| 1/1 [00:00<00:00,  1.08it/s]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         4

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

